## COMPILED DATASETS FOR EV AND ICE VEHICLE EMISSIONS ANALYSIS

This notebook compiles publicly available datasets used for analyzing emissions from passenger electric vehicles (EVs)  and internal combustion engine (ICE) vehicles.

## IMPORTS

In [101]:
import numpy as np
import pandas as pd
import path
import matplotlib.pyplot as plt

In [102]:
# Included to support accessing the variables and dataframes in this notebook from another notebook
import os
from datetime import datetime, timedelta

In [103]:
# Included to ignore any warning dialoge generated
import warnings
warnings.filterwarnings('ignore')

In [104]:
# Added to show all columns in df display
pd.options.display.max_columns = None 

## GENERAL VEHICLE DATA

**VEH DATASET 1: Historic Vehicle Sales Data (1976 to 2023)** \
_U.S. Bureau of Economic Analysis, Light Weight Vehicle Sales in the US (thousand units), Not Seasonally Adjusted (LTOTALNSA) \
and Total Vehicle Sales in the US (thousand units), Not Seasonally Adjusted (TOTALNSA) \
Documentation available here: https://fred.stlouisfed.org/series/LTOTALNSA and https://fred.stlouisfed.org/series/TOTALNSA_

In [105]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
total_veh_sales_US = pd.read_csv("Resources/TOTALNSA.csv", parse_dates=['DATE'])
light_veh_sales_US = pd.read_csv("Resources/LTOTALNSA.csv", parse_dates=['DATE'])
light_veh_car_sales_US = pd.read_csv("Resources/LAUTONSA.csv", parse_dates=['DATE'])
light_veh_truck_sales_US = pd.read_csv("Resources/LTRUCKNSA.csv", parse_dates=['DATE'])
heavy_veh_truck_sales_US = pd.read_csv("Resources/HTRUCKSNSA.csv", parse_dates=['DATE'])

total_veh_sales_US = (total_veh_sales_US
                     .merge(light_veh_sales_US, on='DATE', how='left')
                      .merge(light_veh_car_sales_US, on='DATE', how='left')
                      .merge(light_veh_truck_sales_US, on='DATE', how='left')
                      .merge(heavy_veh_truck_sales_US, on='DATE', how='left')
                     )
                      

total_veh_sales_US = total_veh_sales_US.rename(columns={'TOTALNSA': 'Total Vehicle Sales',
                                                        'LTOTALNSA': 'Light Veh',
                                                        'LAUTONSA': 'Light Veh - Car',
                                                       'LTRUCKNSA': 'Light Veh - Truck',
                                                        'HTRUCKSNSA': 'Heavy Veh'
                                                       })


# Aggregates monthly sales data into annual sales data
total_veh_sales_US = total_veh_sales_US.set_index('DATE')
annual_veh_sales = total_veh_sales_US.resample('Y').sum()
annual_veh_sales = annual_veh_sales.reset_index()
annual_veh_sales['DATE'] = annual_veh_sales['DATE'].dt.year

annual_veh_sales['Light Veh Share - Car (%)'] = annual_veh_sales['Light Veh - Car'] / annual_veh_sales['Light Veh']
annual_veh_sales['Light Veh Share - Truck (%)'] = annual_veh_sales['Light Veh - Truck'] / annual_veh_sales['Light Veh']


# Display the first few rows of the result
display('annual_veh_sales (thousand units, 1976 to 2023)') 
display(annual_veh_sales.head(5))

'annual_veh_sales (thousand units, 1976 to 2023)'

,DATE,Total Vehicle Sales,Light Veh,Light Veh - Car,Light Veh - Truck,Heavy Veh,Light Veh Share - Car (%),Light Veh Share - Truck (%)
0,1976,13294.5,12969.8,9994.0,2975.8,324.7,0.770559,0.229441
1,1977,14859.0,14481.9,11046.0,3435.9,377.1,0.762745,0.237255
2,1978,15420.8,14981.0,11164.0,3817.0,439.8,0.745211,0.254789
3,1979,14148.5,13758.4,10558.8,3199.6,390.1,0.767444,0.232556
4,1980,11469.2,11197.5,8981.8,2215.7,271.7,0.802125,0.197875


**VEH DATASET 2: Fuel Efficiency Standards by Year (1978-2031)** \
_Corporate Average Fuel Economy (CAFE) standards in miles per gallon (mpg) provided by National Highway Traffic Safety Administration
and US EPA \
Documentation available at: https://afdc.energy.gov/data/10562_

In [106]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
vehicle_emission_standards = pd.read_csv("Resources/vehicle_efficiency_CAFE_requirements.csv")

# Display the first 5 rows
vehicle_emission_standards.fillna(0,inplace=True)

# Display the first 5 rows
display('vehicle_emission_standards (mpg, 1978 to 2031)') 
display(vehicle_emission_standards.head(5))

'vehicle_emission_standards (mpg, 1978 to 2031)'

,Model Year,Passenger Cars,Light-Duty Trucks
0,1978,18.0,0.0
1,1979,19.0,0.0
2,1980,20.0,0.0
3,1981,22.0,0.0
4,1982,24.0,17.5


**VEH DATASET 3: Estimated Real-World Fuel Economy, CO2 Emissions and Vehicle Attributes** \
_2023 EPA Automotive Trends Report data in US from 1975 to 2023\
Provides data for supporting fuel efficency predictions above minimum standards and beyond 2031 \
Documentation available here: https://www.epa.gov/automotive-trends/about-automotive-trends-data_

In [107]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
real_world_emissions = pd.read_csv("Resources/estimated_real_world_fuel_economy (1975 to 2022).csv", parse_dates=['Model Year'])

# Replaces missing values with 0 and filters for all vehicle emissions
real_world_emissions.fillna(0,inplace=True)

#Filters emissions for all (aggreggated) type and drops unndeded columns
real_world_emissions_all = real_world_emissions[(real_world_emissions['Vehicle Type'] == 'All')].copy().reset_index(drop=True)

real_world_emissions.drop(columns = ['Production Share', 'Real-World MPG_City', 'Real-World MPG_Hwy',
                                         'Real-World CO2_City (g/mi)', 'Real-World CO2_Hwy (g/mi)',
                                         'Weight (lbs)','Horsepower (HP)', 'Footprint (sq. ft.)'],
                                        axis = 1, inplace = True)


# Display the first 5 rows
display('real_world_emissions, 1975 to 2022')
display(real_world_emissions.head(5))

'real_world_emissions, 1975 to 2022'

,Model Year,Regulatory Class,Vehicle Type,Real-World MPG,Real-World CO2 (g/mi)
0,1975,All,All,13.05970,680.59612
1,1975,Car,All Car,13.45483,660.63740
2,1975,Car,Sedan/Wagon,13.45833,660.46603
3,1975,Truck,All Truck,11.63431,763.86134
4,1975,Truck,Pickup,11.91476,745.88139


In [108]:
# Step 1: Clean the 'Model Year' column
def clean_year(year):
    if isinstance(year, str) and 'Prelim.' in year:
        return year.split()[-1]  # Extract the year from 'Prelim. 2023'
    return str(year)  # Convert all years to string for consistency

real_world_emissions['Model Year'] = real_world_emissions['Model Year'].apply(clean_year)

# Step 2: Create a new column 'Class' based on 'Regulatory Class'
real_world_emissions['Class'] = real_world_emissions['Regulatory Class'].replace({'Car': 'Car', 'Truck': 'Truck', 'All': 'All'})

# Step 3: Group by 'Model Year' and 'Class', and calculate mean values
emissions_by_class = real_world_emissions.groupby(['Model Year', 'Class']).agg({
    'Real-World MPG': 'mean',
    'Real-World CO2 (g/mi)': 'mean'
}).reset_index()

# Step 4: Sort the dataframe by 'Model Year' and 'Class'
emissions_by_class = emissions_by_class.sort_values(['Model Year', 'Class'])

# Step 5: Convert 'Model Year' to datetime
emissions_by_class['Model Year'] = pd.to_datetime(emissions_by_class['Model Year'], format='%Y')

# Optional: If you want to keep only the year part as an integer
emissions_by_class['Model Year'] = emissions_by_class['Model Year'].dt.year

# Display the result
display('emissions_by_class , 1975 to 2023')
display(emissions_by_class.head(10))



'emissions_by_class , 1975 to 2023'

,Model Year,Class,Real-World MPG,Real-World CO2 (g/mi)
0,1975,All,13.059700,680.596120
1,1975,Car,12.680817,706.542443
2,1975,Truck,11.418960,779.081920
3,1976,All,14.221360,625.022380
4,1976,Car,13.436917,678.635883
5,1976,Truck,12.060400,737.305565
6,1977,All,15.067430,589.998800
7,1977,Car,14.447090,623.759743
8,1977,Truck,13.050228,681.629527
9,1978,All,15.837770,561.624420


## ELECTRIC VEHICLE DATA

**EV DATASET 1: Historical Global EV Stock (2013 to 2023)** \
_Contains share (in millions of units) of EV stock in US, Europe, China, and rest of world from 2013 to 2023
Documentation available at: https://www.iea.org/data-and-statistics/charts/global-electric-car-stock-2013-2023_



In [109]:
# Reads in global BEV/PHEV stock
BEV_PHEV_stock= pd.read_csv("Resources/global-electric-car-stock-2013-2023.csv",
                                     delimiter =';',
                                     skiprows=3,
                                     header=0)

BEV_PHEV_stock = BEV_PHEV_stock.rename(columns={'Unnamed: 0': 'Year'})

display('BEV_PHEV_stock (millions of units), 2013 to 2023')
display(BEV_PHEV_stock.head(5))

'BEV_PHEV_stock (millions of units), 2013 to 2023'

,Year,China BEV,China PHEV,Europe BEV,Europe PHEV,United States BEV,United States PHEV,Rest of the world BEV,Rest of the world PHEV
0,2013,0.0,0.0,0.1,0.0,0.1,0.1,0.1,0.0
1,2014,0.1,0.0,0.1,0.1,0.1,0.2,0.1,0.0
2,2015,0.2,0.1,0.2,0.2,0.2,0.2,0.1,0.1
3,2016,0.5,0.2,0.3,0.3,0.3,0.3,0.1,0.1
4,2017,0.9,0.3,0.4,0.4,0.4,0.4,0.2,0.1


#### **EV DATASET 2: Historical Global EV Sales (2012 to 2024)** \
_Contains share (in millions of units) of EV sales in US, Europe, China, and rest of world from 2013 to 2023 \
Documentation available at: https://www.iea.org/data-and-statistics/charts/electric-car-sales-2012-2024_


In [110]:
# Reads in global BEV/PHEV sales
global_EV_sales= pd.read_csv("Resources/electric-car-sales-2012-2024.csv",
                                     delimiter =';',
                                     skiprows=3,
                                     header=0)

global_EV_sales = global_EV_sales.rename(columns={'Unnamed: 0': 'Year'})

display('global_EV_sales (millions of units), 2012 to Q1 2024')
display(global_EV_sales.head(5))

'global_EV_sales (millions of units), 2012 to Q1 2024'

,Year,China,Europe,United States,Rest of the world
0,2012,0.0,0.0,0.1,0.0
1,2013,0.0,0.1,0.1,0.0
2,2014,0.1,0.1,0.1,0.0
3,2015,0.2,0.2,0.1,0.0
4,2016,0.3,0.2,0.2,0.0


**EV DATASET 3: Historic and Projected (2011 to 2035) EV Sales by Powertrain Type** \
_Includes historical and projected data aligned to stated policies scenario (STEPS) and announced pledges scenario (APS) \
for electric vehicles sales, stock, charging infrastructure and oil displacement \
Documentation available at: https://www.iea.org/data-and-statistics/data-product/global-ev-outlook-2024#global-ev-data_

In [111]:
# Reads in historic and projeted EV electric demand data and joins datafames
ev_outlook= pd.read_csv("Resources/IEA Global EV Data 2024.csv")

# Historic EV Outlook data for US
ev_outlook_US_hist = ev_outlook[(ev_outlook['category'] == 'Historical') & (ev_outlook['region'] == 'USA') & (ev_outlook['mode'] == 'Cars')].copy().reset_index(drop=True)

# Projected EV Outlook data based on stated policies for US
ev_outlook_US_proj_STEPS = ev_outlook[(ev_outlook['category'] == 'Projection-STEPS') & (ev_outlook['region'] == 'USA') & (ev_outlook['mode'] == 'Cars')].copy().reset_index(drop=True)

# Projected EV Outlook data based on announced pledges for US
ev_outlook_US_proj_APS = ev_outlook[(ev_outlook['category'] == 'Projection-APS') & (ev_outlook['region'] == 'USA') & (ev_outlook['mode'] == 'Cars')].copy().reset_index(drop=True)

display('ev_outlook_US_hist (millions of units, %), 2011 to 2035')
display(ev_outlook_US_hist)

display('ev_outlook_US_proj_STEPS (millions of units, %), 2011 to 2035')
display(ev_outlook_US_proj_STEPS)

display('ev_outlook_US_proj_APS (millions of units, %), 2011 to 2035')
display(ev_outlook_US_proj_APS)

'ev_outlook_US_hist (millions of units, %), 2011 to 2035'

,region,category,parameter,mode,powertrain,year,unit,value
0,USA,Historical,EV sales,Cars,BEV,2010,Vehicles,1200.00000
1,USA,Historical,EV stock,Cars,BEV,2010,Vehicles,3800.00000
2,USA,Historical,Oil displacement Mbd,Cars,EV,2010,Milion barrels per day,0.00016
3,USA,Historical,"Oil displacement, million lge",Cars,EV,2010,"Oil displacement, million lge",9.40000
4,USA,Historical,Electricity demand,Cars,EV,2010,GWh,96.00000
...,...,...,...,...,...,...,...,...
139,USA,Historical,EV sales,Cars,FCEV,2023,Vehicles,3000.00000
140,USA,Historical,EV stock,Cars,FCEV,2023,Vehicles,18000.00000
141,USA,Historical,EV sales,Cars,PHEV,2023,Vehicles,290000.00000
142,USA,Historical,"Oil displacement, million lge",Cars,EV,2023,"Oil displacement, million lge",9700.00000


'ev_outlook_US_proj_STEPS (millions of units, %), 2011 to 2035'

,region,category,parameter,mode,powertrain,year,unit,value
0,USA,Projection-STEPS,Oil displacement Mbd,Cars,EV,2020,Milion barrels per day,0.043
1,USA,Projection-STEPS,"Oil displacement, million lge",Cars,EV,2020,"Oil displacement, million lge",2500.000
2,USA,Projection-STEPS,Electricity demand,Cars,EV,2020,GWh,5400.000
3,USA,Projection-STEPS,EV sales,Cars,BEV,2020,Vehicles,230000.000
4,USA,Projection-STEPS,EV stock,Cars,BEV,2020,Vehicles,1100000.000
...,...,...,...,...,...,...,...,...
72,USA,Projection-STEPS,Oil displacement Mbd,Cars,EV,2035,Milion barrels per day,2.500
73,USA,Projection-STEPS,"Oil displacement, million lge",Cars,EV,2035,"Oil displacement, million lge",140000.000
74,USA,Projection-STEPS,Electricity demand,Cars,EV,2035,GWh,460000.000
75,USA,Projection-STEPS,EV stock share,Cars,EV,2035,percent,36.000


'ev_outlook_US_proj_APS (millions of units, %), 2011 to 2035'

,region,category,parameter,mode,powertrain,year,unit,value
0,USA,Projection-APS,EV sales,Cars,BEV,2020,Vehicles,230000.0
1,USA,Projection-APS,EV stock,Cars,BEV,2020,Vehicles,1100000.0
2,USA,Projection-APS,EV sales,Cars,FCEV,2020,Vehicles,1200.0
3,USA,Projection-APS,EV stock,Cars,FCEV,2020,Vehicles,9200.0
4,USA,Projection-APS,EV sales,Cars,PHEV,2020,Vehicles,64000.0
...,...,...,...,...,...,...,...,...
72,USA,Projection-APS,EV stock,Cars,BEV,2035,Vehicles,76000000.0
73,USA,Projection-APS,EV stock,Cars,FCEV,2035,Vehicles,130000.0
74,USA,Projection-APS,EV stock,Cars,PHEV,2035,Vehicles,14000000.0
75,USA,Projection-APS,Oil displacement Mbd,Cars,EV,2035,Milion barrels per day,2.0


**EV DATASET 4: Compariative Lifecycle GHG emission for Midsized BEV and ICE vehicle**
_Compares life-cycle greenhouse gas emissions of a mid-size BEV to ICE vehicle \
Considers vehicle manufacturing, battery assembly, battery mineral mining, electricity and fuel cycle well-to-wheels emissions \
Data measued in tCO2e per vehicle lifetime and sourced from IEA / Argonne National Lab analysis conducted 2020 \
Documentation available at: https://www.iea.org/data-and-statistics/charts/comparative-life-cycle-greenhouse-gas-emissions-of-a-mid-size-bev-and-ice-vehicle_



In [112]:
# Reads in lifecycle emissions data for BEV and ICE
BEV_and_ICE_lifecycle_GHG= pd.read_csv("Resources/comparative-life-cycle-greenhouse-gas-emissions-of-a-mid-size-bev-and-ice-vehicle.csv",
                                     skiprows=3,
                                     header=0)

BEV_and_ICE_lifecycle_GHG = BEV_and_ICE_lifecycle_GHG.rename(columns={'Unnamed: 0': 'Vehicle Type'})

display('BEV_and_ICE_lifecycle_GHG (tCO2e per vehicle)')
display(BEV_and_ICE_lifecycle_GHG)

'BEV_and_ICE_lifecycle_GHG (tCO2e per vehicle)'

,Vehicle Type,Vehicle manufacturing,Batteries-assembly and other,Batteries-minerals,Electricity,Fuel cycle (well-to-wheel),Unnamed: 6
0,Battery electric vehicle - Base case,5.4,1.20,1.40,11.7,0.0,NaN
1,Battery electric vehicle - High-GHG minerals ...,5.4,1.20,2.80,11.7,0.0,NaN
2,Internal combustion engine vehicle,6.0,0.01,0.03,0.0,35.9,NaN


## ELECTRICITY DATA

**ELEC DATASET 1: US Electricity Generation By Fuel Type** \
_EIA net electricity generation for all sectors energy in US from 2010 to 2023 \
Documentation available at: link to be added_

In [113]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
net_generation = pd.read_csv("Resources/Net_generation_for_all_sectors.csv",
                                   skiprows=4,
                                   header=0)

# Replaces missing values with 0
net_generation.fillna(0,inplace=True)

# Drops uneeded columns
net_generation.drop(columns="source key", axis=1, inplace=True)
net_generation['description'] = net_generation['description'].str.replace('United States : ', '', regex=False)
net_generation = net_generation.iloc[2:]
net_generation = net_generation.reset_index(drop=True)
net_generation = net_generation.set_index('description')
net_generation.index.name = None

# Transposes dataframe to show records by year to align with other datasets
net_generation = net_generation.T
net_generation = net_generation.reset_index()
net_generation = net_generation.rename(columns={'index': 'year'})
net_generation.drop(0, inplace=True)

# Display the first 5 rows
display('net_generation (thousand MWh), 2010 to 2023')
display (net_generation.head(5))

'net_generation (thousand MWh), 2010 to 2023'

,year,all fuels (utility-scale),coal,petroleum liquids,petroleum coke,natural gas,other gases,nuclear,conventional hydroelectric,other renewables,wind,all utility-scale solar,geothermal,biomass,wood and wood-derived fuels,other biomass,hydro-electric pumped storage,other,all solar,small-scale solar photovoltaic,all utility-scale solar
1,2010,4125060,1847290,23337,13724,987697,11313,806968,260203,0,94652,1212,15219,56089,37172,18917,-5501,12855,--,--,1212
2,2011,4100141,1733430,16086,14096,1013689,11566,790204,319355,0,120177,1818,15316,56671,37449,19222,-6421,14154,--,--,1818
3,2012,4047765,1514043,13403,9787,1225894,11898,769331,276240,0,140822,4327,15562,57622,37799,19823,-4950,13787,--,--,4327
4,2013,4065964,1581115,13820,13344,1124836,12853,789016,268565,0,167840,9036,15775,60858,40028,20830,-4681,13588,--,--,9036
5,2014,4093564.0,1581710.0,18276.0,11955.0,1126635.0,12022.0,797166.0,259367.0,0.0,181655.0,17691.0,15877.0,63989.0,42340.0,21650.0,-6174.0,13393.0,28924.0,11233.0,17691.0


**ELEC DATASET 2: US Electricity Consumption By Fuel Type** \
_EIA electricity consumption for all energy sectors in US from 2010 to 2023\
Documentation available at: link to be added_

In [114]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
sector_elec_consumption = pd.read_csv("Resources/Consumption_for_electricity_generation_for_all_sectors.csv",
                                   skiprows=4,
                                   header=0)

# Replaces missing values with 0
sector_elec_consumption.fillna(0,inplace=True)

# Removes uneeded data
sector_elec_consumption.drop(columns="source key", axis=1, inplace=True)
sector_elec_consumption['description'] = sector_elec_consumption['description'].str.replace('United States : ', '', regex=False)
sector_elec_consumption = sector_elec_consumption.iloc[2:]
sector_elec_consumption = sector_elec_consumption.reset_index(drop=True)
sector_elec_consumption = sector_elec_consumption.set_index('description')
sector_elec_consumption.index.name = None

# Transposes dataframe to show records by year to align with other datasets
sector_elec_consumption = sector_elec_consumption.T
sector_elec_consumption = sector_elec_consumption.reset_index()
sector_elec_consumption = sector_elec_consumption.rename(columns={'index': 'year'})
sector_elec_consumption.drop(0, inplace=True)

# Display the first 5 rows
display('sector_elec_consumption (thousand tons coal, thousand barrels liquid, thousand tons coke, thousand Mcf NG), 2010 to 2023')
display(sector_elec_consumption.head(5))

'sector_elec_consumption (thousand tons coal, thousand barrels liquid, thousand tons coke, thousand Mcf NG), 2010 to 2023'

,year,coal,petroleum liquids,petroleum coke,natural gas
1,2010,979684.0,40103.0,4994.0,7680185.0
2,2011,934938.0,27326.0,5012.0,7883865.0
3,2012,825734.0,22604.0,3675.0,9484710.0
4,2013,860729.0,23231.0,4852.0,8596299.0
5,2014,853634.0,31531.0,4412.0,8544387.0


## US GHG Emissions Datasets

**GHG DATASET 1: Emissions by Sector** \
_EPA US Greenhouse Gas Inventory by Economic Sector, MMT CO2 eq.\
Documentation available at https://afdc.energy.gov/data/10802_

In [115]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
emissions_by_econ_sector = pd.read_csv("Resources/GHG_emissions_by_econ_sector.csv",
                                       skiprows=2,
                                       header=0)
                                      
# Drop unnamed columns
emissions_by_econ_sector = emissions_by_econ_sector.drop(columns=[col for col in emissions_by_econ_sector.columns if col.startswith('Unnamed')])

# Converts data columns to numeric
columns_to_convert = ['Transportation', 'Electricity Generation', 'Industry', 'Total']
emissions_by_econ_sector[columns_to_convert] = emissions_by_econ_sector[columns_to_convert].replace(',', '', regex=True).astype(float)

# Display the first 5 rows and data types
display('emissions_by_econ_sector (MMT CO2e), 1990 to 2022')
display(emissions_by_econ_sector.head(5))

'emissions_by_econ_sector (MMT CO2e), 1990 to 2022'

,Year,Transportation,Electricity Generation,Industry,Agriculture,Commercial,Residential,Total
0,1990,1521.4,1880.2,1723.3,595.9,447.0,345.6,6513.5
1,1991,1474.8,1874.4,1702.9,587.4,454.5,355.3,6449.2
2,1992,1533.8,1889.6,1729.3,587.5,450.0,361.8,6552.0
3,1993,1570.2,1965.0,1701.0,608.7,443.1,373.1,6661.2
4,1994,1624.5,1989.6,1719.7,612.2,446.1,363.8,6755.8


In [116]:
# # Sets index to Year
# emissions_by_econ_sector = emissions_by_econ_sector.set_index('Year')

# # Select the 2022 GHG data
# data_2022 = emissions_by_econ_sector.loc['2022']
# sectors = ['Transportation', 'Electricity Generation', 'Industry', 'Agriculture', 'Commercial', 'Residential']
# values = data_2022[sectors]

# # Creates pie chart 
# plt.figure(figsize=(10, 8))
# plt.pie(values, labels=sectors, autopct='%1.1f%%', startangle=90)
# plt.title('GHG Emissions by Economic Sector in 2022')
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
# plt.show()

**GHG DATASET 2: Net Avoided Emissions and Avoided Share (2023 to 2035)** \
_IEA projection of net avoided emissions from BEV and ICE using lifecycle GHG values (measured in millions of tons CO2e)\
and share of net avoided emissions (measured in percentages) for stated policies scenario (STEPS), announced pledges scenario (APS) \
and Net Zero Emissions (NZE) \
Documentation available at: https://www.iea.org/reports/global-ev-outlook-2024/outlook-for-emissions-reductions#abstract_

In [117]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
avoided_emissions_2035_proj = pd.read_csv("Resources/net-avoided-well-to-wheel-greenhouse-gas-emissions-from-electric-vehicle-deployment-2023-2035.csv",
                                          delimiter=';',
                                          skiprows=3,
                                          header=0)

avoided_emissions_2035_proj = avoided_emissions_2035_proj.rename(columns={'Unnamed: 0': 'Year'})


# Display the first 5 rows and data types
display('avoided_emissions_2035_proj (MMT CO2e), 2023 to 2035')
display(avoided_emissions_2035_proj.head(5))

'avoided_emissions_2035_proj (MMT CO2e), 2023 to 2035'

,Year,STEPS,APS,NZE
0,2023,-130,-130,-120
1,2024,-180,-180,-200
2,2025,-240,-250,-310
3,2026,-320,-330,-440
4,2027,-420,-430,-600


In [118]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
share_avoided_emissions_2035_proj = pd.read_csv("Resources/net-avoided-well-to-wheel-share-of-avoided-emissions-by-mode-2023-2035.csv",
                                          delimiter=';',
                                          skiprows=3,
                                          header=0)

share_avoided_emissions_2035_proj = share_avoided_emissions_2035_proj.rename(columns={'Unnamed: 0': 'Scnenario'})


# Display the first 5 rows and data types
share_avoided_emissions_2035_proj.head(5)


,Scnenario,LDVs,2/3Ws,Buses,Trucks
0,STEPS,81,4,10,6
1,APS,77,5,12,6
2,NZE,67,11,14,8


**GHG DATASET 3: Average Emissions Lifecycle by Powertrain Type (2023-2035)** \
Compares global average lifecycle emissions by powertrain in the Stated Policies and Announced Pledges Scenarios, 2023-2035\
Documentation available at: https://www.iea.org/data-and-statistics/charts/comparison-of-global-average-lifecycle-emissions-by-powertrain-in-the-stated-policies-and-announced-pledges-scenarios-2023-2035

In [119]:
# Reads in lifecycle emissions data for each pawertrain type (CO2-eq/vehicle)
lifecycle_GHG_2023_2035= pd.read_csv("Resources/comparison-of-global-average-lifecycle-emissions-by-powertrain-in-the-stated-policies-and-announced-pledges-scenarios-2023-2035.csv",
                                    delimiter=";", 
                                     skiprows=3,
                                     header=0)

lifecycle_GHG_2023_2035 = lifecycle_GHG_2023_2035 .rename(columns={'Unnamed: 0': 'Powertrain Type'})

display ('lifecycle_GHG_2023_2035 (t CO2e per vehicle), 2023 to 2035')
display (lifecycle_GHG_2023_2035.head(5))

'lifecycle_GHG_2023_2035 (t CO2e per vehicle), 2023 to 2035'

,Powertrain Type,Car production,Battery production,Well-to-tank,Tank-to-wheel,Grid decarbonisation impact
0,ICEV,3.7,0.0,11.5,30.9,0.0
1,HEV,3.7,0.1,9.8,26.3,0.0
2,PHEV,4.4,1.3,11.7,15.8,1.9
3,BEV,3.3,5.3,14.5,0.0,4.8
4,ICEV,3.7,0.0,8.1,26.4,0.0


## Misc Datasets

**Misc DATASET 1: Coal vs EV Emissions Differences By Country (2019 to 2021)** \
_dataset source and description_

In [120]:
# Reads the CSV file, skipping the first 3 rows and using the 4th row as header
coal_vs_EV_emissions = pd.read_csv("Resources/coal_power_vs_ev_emissions_with_difference.csv")
coal_vs_EV_emissions = coal_vs_EV_emissions.sort_values(by=['Country', 'Year'])

# Display the first 5 rows and USA group
coal_vs_EV_emissions.tail(3)

,Country,Year,Coal Power Emissions (metric tons CO2),Number of EVs,Emissions Saved per EV (metric tons CO2/year),Total Emissions Saved by EVs (metric tons CO2),Difference (Coal Emissions - EV Savings) (metric tons CO2)
7,United States,2019,1300000000,1200000,4.6,5520000,1294480000
2,United States,2020,1200000000,1700000,4.6,7820000,1192180000
8,United States,2021,1100000000,2000000,4.6,9200000,1090800000


## Dataset Use

**EXTRACTING DATAFRAMES FOR USE** \
The %run magic command may be added to other .ipynb files to execute the this notebook

In [121]:
# EXECUTED FROM NEW NOTEBOOK OPTION
    # Runs all the code in vehicle_fuel_emissions_data.ipynb, its variables and dataframes available in other notebook
    # To use this option, paste the "%run vehicle_fuel_emissions_data" command into a notebook that will execute this notebook

# %run vehicle_fuel_emissions_data.ipynb 